# Final real run on 22.03.24

Tuned MinKNOW progressive unblock durations

now with real SARS2 samples (the poly-adenylation may have resulted in lots of side products)

on same Minion flow cell, now channels 201-400

In [1]:
from concurrent.futures import ThreadPoolExecutor
import numpy
import sys
sys.path.append("../../external/read_until_api")
import read_until
import os
read_until.__version__


'3.4.1'

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio
pio.templates.default = "simple_white"
colorscale = px.colors.sequential.Plasma
colorscale = [colorscale[0]] + colorscale[3:8]


#default renderer (svg means very small file size, visibility on github, high quality, but requires sometimes setting height and width)
pio.renderers.default = "jupyterlab"

px.defaults.width=1000
px.defaults.height=500

#fig.update_traces(marker_line_width=0.2)

#set default template as "simple_white" (no background, no grid lines)
pio.templates["simple_white"].layout["yaxis"]["showgrid"] = True
pio.templates["simple_white"].layout["xaxis"]["showgrid"] = True
pio.templates.default = "simple_white"

In [3]:
from time import time, sleep
import bottleneck
import numpy as np

In [4]:
from minknow_api.manager import Manager

def get_connected_devices(host = "127.0.0.1", port = None):
    manager = Manager(host=host, port=port)
    positions = {}
    for position in manager.flow_cell_positions():
        positions[position.name] = position
    return positions

def get_device_port(device_name, host = "127.0.0.1", port = None):
    try:
        device_port = get_connected_devices(host, port)[device_name].rpc_ports.secure
        print(f"Device {device_name} has port {device_port}.")
        return device_port
    except:
        print("Could not find device. Is it connected and is the minknow_api address correct?")

In [5]:
get_connected_devices()

{'MN41718': FlowCellPosition('127.0.0.1', {name: "MN41718"
 state: STATE_RUNNING
 rpc_ports {
   secure: 8004
   secure_grpc_web: 8006
 }
 protocol_state: PROTOCOL_RUNNING
 parent_name: "MN41718"
 }),
 'P2S-01245-A': FlowCellPosition('127.0.0.1', {name: "P2S-01245-A"
 state: STATE_HARDWARE_REMOVED
 rpc_ports {
   secure: 8000
   secure_grpc_web: 8001
 }
 error_info: "The sequencing device was unplugged"
 protocol_state: PROTOCOL_FINISHED_SUCCESSFULLY
 device_type: PROMETHION
 parent_name: "P2S-01245"
 }),
 'P2S-01245-B': FlowCellPosition('127.0.0.1', {name: "P2S-01245-B"
 state: STATE_HARDWARE_REMOVED
 rpc_ports {
   secure: 8002
   secure_grpc_web: 8003
 }
 error_info: "The sequencing device was unplugged"
 device_type: PROMETHION
 parent_name: "P2S-01245"
 })}

In [6]:
sys.path.append("../../")
import warpdemux

In [7]:
from warpdemux.live_balancing.entry_point import Session
# add read_until proper submodule

## with updated cache

In [8]:
def setup_read_until_client(mk_port, first_channel, last_channel, filter_strands = True, prefilter_classes = {"adapter", "polya"}):
    return read_until.ReadUntilClient(
        mk_port=mk_port,
        one_chunk=False,
        cache_type=read_until.AccumulatingCache,
        max_raw_signal = 30000,
        first_channel = first_channel,
        last_channel = last_channel,
        calibrated_signal=True,
        filter_strands = filter_strands, 
        prefilter_classes = prefilter_classes
    )

In [9]:
import time

In [10]:
# changed prefilter to false
read_until_client = setup_read_until_client(get_device_port("MN41718"), 201, 400, filter_strands = True, prefilter_classes = {"adapter"})
read_until_client.reset()

Device MN41718 has port 8004.


In [11]:
analysis = Session(
    read_until_client,
    config_path = "24_03_22_balancing_minion_half.toml"
)

[[balancers]]
num_bcs = 12
balance_threshold = 0.05
min_stat = 100
balance_type = "none"
name = "none"
pred_conf_threshold = 0.2
channel_num = 40
watch_for_missing = false
wait_to_see = 900
blacklist = []
ignorelist = []
max_stats = [ "inf", "inf", "inf", "inf", "inf", "inf", "inf", "inf", "inf", "inf", "inf", "inf",]
channels = [ 201, 205, 209, 214, 216, 220, 228, 229, 239, 254, 259, 263, 264, 265, 271, 275, 277, 282, 287, 294, 305, 307, 313, 316, 322, 330, 333, 339, 341, 351, 353, 355, 357, 361, 368, 373, 376, 388, 393, 400,]

[[balancers]]
num_bcs = 12
balance_threshold = 0.1
min_stat = 500
balance_type = "read_count"
name = "read_balancing"
pred_conf_threshold = 0.2
channel_num = 40
pod5_watch_dir = "E:/Nanopore Data/2024_03_22_Run_127_WDX_live_8_SARS2/2024_03_22_Run_127_WDX_live_SARS2/no_sample/20240322_1943_MN41718_FAX70708_bcee8f90/pod5_pass"
pod5_check_interval = 1
watch_for_missing = false
wait_to_see = 900
blacklist = []
ignorelist = []
max_stats = [ "inf", "inf", "inf", "inf

# started at min 18

In [ ]:
analysis.run(batch_size=1000)

In [13]:
analysis.stop()

--------------------------------------
Balancer: none (none)
--------------------------------------

[2024-03-22_20-02-04] --- 10:22:43 

Processed reads
Accepted Classified: 30467 / 33538
Accepted Unclassified: 1746 / 33538
Accepted Failed: 0 / 33538
Accepted Noise: 1325 / 33538
Rejected Classified: 0 / 33538
Rejected Unclassified: 0 / 33538
Rejected Failed: 0 / 33538
Rejected Noise: 0 / 33538

Processed reads; #Accepted per BC
Fraction represents accepted processed reads per bc / processed reads
00        01        02        03        04        05        06        07        08        09        10        11       
1701      1244      2860      6872      5853      157       1453      921       2934      3385      2987      100      
0.06      0.04      0.09      0.23      0.19      0.01      0.05      0.03      0.10      0.11      0.10      0.00     

Processed reads; #Rejected per BC
Fraction represents rejected processed reads per bc / processed reads
00        01        02        03

# stopped at 14 h 40 min